In [1]:
# test
# %run './base.ipynb'

## Preprocessing
- impute (various kinds)/dummy values/drop missing values
- outlier removal?
- Scale (standardize)/don't scale 
- feature selection (PCA, etc.)/none

## Models
- Linear Regression (Anna)
- SVR (Moritz)
- GradientBoostingRegressor (David)

## Params
- cv=10
- scoring=['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error']
- Best params wählen nach MSE, aber auch r^2 notieren für Vergleichbarkeit

## AutoML
- 

## Datasets
- Bike sharing (Kaggle) ()
    - https://www.kaggle.com/c/184702-tu-ml-ws-18-bike-sharing#_=_
    - large samples (train = 8690), small dimension (15)
    - attribute characteristics: numeric, date?
- Student performance (Kaggle) (Moritz)
    - https://www.kaggle.com/c/184702-tu-ml-ws-18-student-performance
    - small samples (train = 198), medium dimension (32)
    - attribute characteristics: numeric, categorical 
- Blog feedback (David)
    - https://archive.ics.uci.edu/ml/datasets/BlogFeedback
    - very large samples (60021), large dimension (281)
    - attribute characteristics: numeric
- Forest fires (Anna)
    - https://archive.ics.uci.edu/ml/datasets/Forest+Fires
    - medium samples (513), small dimension (13) 
    - attribute characteristics: numeric
    
## Steps
- Imports for all Datasets
- functions for all Regressors

In [ ]:
# Preprocessing


In [2]:
# Linear Regression


In [3]:
# SVR


In [4]:
# Gradient Boosting
